In [1]:
!pip install pyDOE
!pip install cma

  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=1998547bc9dd9ec8ae1e5db4584f0402a1e12ae32c5cfc328fe5a1411a73f70b
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 2.7MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F20 '''
def F20(X):
    f = bn.F20()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 30.57 ,random_state=0 , l1_ratio=1.0, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 37.213462 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_204000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_20_800Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [5]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,201):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 200, [5] * 200 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 200000)

n_obs , dim =  30, 200
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([10,600])
for i in range(0, 10):
  print ('Run : '+ str(i))
  min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=269407, Fri Apr 17 20:34:31 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 1.723099660867862e+06 1.0e+00 2.44e+00  2e+00  2e+00 0:00.1
    2     38 1.574812660040495e+06 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [1967341.94391646] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [1646648.63691494] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    3     57 1.535474336560592e+06 1.0e+00 2.34e+00  2e+00  2e+00 0:00.3
   33    627 2.974905862633111e+05 1.1e+00 2.05e+00  2e+00  2e+00 0:03.3
   73   1387 5.255221616173489e+04 1.2e+00 1.88e+00  2e+00  2e+00 0:07.3
  100   1900 -3.624494527546344e+05 1.2e+00 1.58e+00  2e+00  2e+00 0:10.0
  161   3059 -1.278472937134683e+06 1.2e+00 8.31e-01  8e-01  8e-01 0:16.1
  200   3800 -1.811605905790438e+06 1.2e+00 5.42e-01  5e-01  5e-01 0:20.0
  280   5320 -2.239419521472581e+06 1.2e+00 2.27e-01  2e-01  2e-01 0:28.0
  300   5700 -2.271944100523960e+06 1.2e+00 1.81e-01  2e-01  2e-01 0:30.0
  400   7600 -2.329857453413310e+06 1.2e+00 6.26e-02  6e-02  6e-02 0:39.6
  500   9500 -2.335879775959103e+06 1.3e+00 2.13e-02  2e-02  2e-02 0:49.3
  600  11400 -2.336654667640978e+06 1.3e+00 7.41e-03  7e-03  7e-03 0:58.9
  700  13300 -2.336753938406459e+06 1.3e+00 2.66e-03  3e-03  3e-03 1:08.7
  800  15200 -2.336765350125268e+06 1.3e+00 8.10e-04  8e-04  8e-04 1:18.4
  900  17100 -2.336766344253893e+06 1.3e+

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [1969341.12838215] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [2341327.92528471] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    2     38 1.279222397997117e+06 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2
    3     57 1.257876864865274e+06 1.0e+00 2.34e+00  2e+00  2e+00 0:00.3


...

    3     57 1.946946995636801e+06 1.0e+00 2.34e+00  2e+00  2e+00 0:00.3
   39    741 4.517696327922235e+05 1.1e+00 2.12e+00  2e+00  2e+00 0:03.3
   87   1653 1.083587973278053e+05 1.2e+00 1.80e+00  2e+00  2e+00 0:07.4
  100   1900 -1.506284630409558e+04 1.2e+00 1.63e+00  2e+00  2e+00 0:08.5
  173   3287 -1.322845926466106e+06 1.2e+00 7.85e-01  8e-01  8e-01 0:14.5
  200   3800 -1.692558175766339e+06 1.2e+00 6.16e-01  6e-01  6e-01 0:16.8
  296   5624 -2.256559437748203e+06 1.3e+00 2.15e-01  2e-01  2e-01 0:24.8
  300   5700 -2.260786460295851e+06 1.3e+00 2.06e-01  2e-01  2e-01 0:25.1
  400   7600 -2.326369762980005e+06 1.3e+00 6.87e-02  7e-02  7e-02 0:33.4
  500   9500 -2.335649780263030e+06 1.3e+00 2.35e-02  2e-02  2e-02 0:41.6
  600  11400 -2.336660001879552e+06 1.3e+00 8.15e-03  8e-03  8e-03 0:49.7
  700  13300 -2.336753836447396e+06 1.3e+00 2.64e-03  3e-03  3e-03 0:58.0
  800  15200 -2.336765069265575e+06 1.3e+00 8.52e-04  8e-04  8e-04 1:06.2
  900  17100 -2.336766271465912e+06 1.3e+

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [1969341.12838215] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


termination on tolfun=1e-11 (Sat Apr 18 01:39:25 2020)
final/bestever f-value = 1.969341e+06 1.969341e+06
incumbent solution: [ 0.72615136  0.31599894  3.77915436  2.02284736 -4.09218194  3.47427043
 -0.98532926 -2.75635507 ...]
std deviations: [2.43541699 2.43510108 2.43536984 2.43508214 2.43557311 2.43501294
 2.43513397 2.43532747 ...]
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=305381, Sat Apr 18 01:39:25 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 2.451696655528771e+06 1.0e+00 2.43e+00  2e+00  2e+00 0:00.2


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [2914091.42621176] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [2552108.16275168] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    2     38 2.289652735451565e+06 1.0e+00 2.38e+00  2e+00  2e+00 0:00.4
    3     57 2.256660210665280e+06 1.0e+00 2.33e+00  2e+00  2e+00 0:00.6
   34    646 6.969142748630636e+05 1.1e+00 2.00e+00  2e+00  2e+00 0:03.7
   79   1501 6.135453258477816e+05 1.2e+00 1.85e+00  2e+00  2e+00 0:07.7
  100   1900 5.847316390540232e+05 1.2e+00 1.74e+00  2e+00  2e+00 0:09.6
  167   3173 2.668110386445906e+05 1.3e+00 1.36e+00  1e+00  1e+00 0:15.7
  200   3800 1.611195701778831e+05 1.3e+00 1.12e+00  1e+00  1e+00 0:18.7
  287   5453 -1.657425180265630e+05 1.3e+00 6.71e-01  7e-01  7e-01 0:26.7
  300   5700 -2.026710704664652e+05 1.3e+00 6.13e-01  6e-01  6e-01 0:27.8
  400   7600 -3.438754905255045e+05 1.3e+00 3.21e-01  3e-01  3e-01 0:36.8
  500   9500 -3.901902035964022e+05 1.3e+00 1.60e-01  2e-01  2e-01 0:45.7
  600  11400 -4.091242237999067e+05 1.4e+00 1.02e-01  1e-01  1e-01 0:54.7
  700  13300 -4.189139387841284e+05 1.7e+00 9.46e-02  9e-02  1e-01 1:03.8
  800  15200 -4.247277771783146e+05 1.8e+00 6

In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('Results\\F20_X_Values_first.csv')
Krig_Fun = np.zeros(10)
SVM_Fun = np.zeros(10)
ELN_Fun = np.zeros(10)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F20(X_Values.iloc[i,:200])
    SVM_Fun [i] = F20(X_Values.iloc[i,200:400])
    ELN_Fun [i] = F20(X_Values.iloc[i,400:600])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([208029.50652926]), count=array([1])) 0.005004202911068852
SVM
ModeResult(mode=array([1599291.56705953]), count=array([1])) 358353.86741827935
ELN
ModeResult(mode=array([64242.26058638]), count=array([1])) 9042.960867471036
